Bank Churning
==============

Imports

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

TF Version

In [3]:
tf.__version__

'2.7.0'

Data Preprocessing
-------------------

Import the Data

In [4]:
data = pd.read_csv('Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Remove Useless Columns

In [5]:
clean = data.drop(columns = ['RowNumber','CustomerId', 'Surname' ])
clean

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Normalize CreditScore, Age, Balance, EstimatedSalary

In [7]:
from sklearn.preprocessing import MinMaxScaler

clean['CreditScore'] = MinMaxScaler().fit_transform(np.array(clean['CreditScore']).reshape(-1,1))
clean['Age'] = MinMaxScaler().fit_transform(np.array(clean['Age']).reshape(-1,1))
clean['Balance'] = MinMaxScaler().fit_transform(np.array(clean['Balance']).reshape(-1,1))
clean['EstimatedSalary'] = MinMaxScaler().fit_transform(np.array(clean['EstimatedSalary']).reshape(-1,1))


In [8]:
x = clean.iloc[:-1, :-1].values
y = clean.iloc[:-1, -1].values

Encode Gender Column

In [9]:
from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
x[:,2] = l.fit_transform(x[:,2])

One-Hot Encode Geography Column

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))


Split Data into Training and Test Set

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

Build ANN
----------------------------------------------------------------

In [17]:
#init ANN
ann = tf.keras.models.Sequential()

#input layer
ann.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))

#hidden layers
ann.add(tf.keras.layers.Dense(units = 4, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 8, activation = 'relu'))


#output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

Training ANN
----------------------------------------------------------------

In [18]:
# Compile ANN
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ann.fit(x_train, y_train, batch_size = 16, epochs= 300)

Epoch 1/300
500/500 [==============================] - 3s 6ms/step - loss: 0.5078 - accuracy: 0.7823
Epoch 2/300
500/500 [==============================] - 3s 5ms/step - loss: 0.4372 - accuracy: 0.7947
Epoch 3/300
500/500 [==============================] - 3s 5ms/step - loss: 0.4204 - accuracy: 0.8082
Epoch 4/300
500/500 [==============================] - 3s 5ms/step - loss: 0.4069 - accuracy: 0.8159
Epoch 5/300
500/500 [==============================] - 3s 5ms/step - loss: 0.3911 - accuracy: 0.8300
Epoch 6/300
500/500 [==============================] - 3s 5ms/step - loss: 0.3724 - accuracy: 0.8446
Epoch 7/300
500/500 [==============================] - 3s 5ms/step - loss: 0.3593 - accuracy: 0.8480
Epoch 8/300
500/500 [==============================] - 3s 5ms/step - loss: 0.3514 - accuracy: 0.8581
Epoch 9/300
500/500 [==============================] - 3s 5ms/step - loss: 0.3460 - accuracy: 0.8584
Epoch 10/300
500/500 [==============================] - 3s 5ms/step - loss: 0.3439 - accura